<a href="https://colab.research.google.com/github/JonathanCornish/SP500_Data/blob/main/S%26P_Earnings_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
# url for NYU Stern professor Aswath Damadoran's page on historical earnings & price level of S&P 500
url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/spearn.htm"

In [8]:
# the below cell wasn't working, so I had to google it & this came up as solution:
# https://exerror.com/urllib-error-urlerror-urlopen-error-ssl-certificate_verify_failed-certificate-verify-failed-unable-to-get-local-issuer-certificate/
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [9]:
# use pandas's pd.read_html() method to read the table on the webpage
table = pd.read_html(url, attrs = {"border": "0"})
# the above line returns a list of tables, but we just want the first element from that list (which happens to be the only item in the list):
table = table[0]

In [10]:
table

,0,1,2,3,4,5,6
0,Year,Earnings Yield,Dividend Yield,S&P 500,Earnings,Dividends,Payout Ratio
1,1960,5.34%,3.41%,58.11,3.10,1.98,63.86%
2,1961,4.71%,2.85%,71.55,3.37,2.04,60.51%
3,1962,5.81%,3.40%,63.1,3.67,2.15,58.52%
4,1963,5.51%,3.13%,75.02,4.13,2.35,56.81%
...,...,...,...,...,...,...,...
58,2017,4.66%,1.83%,2673.61,124.51,48.93,39.30%
59,2018,5.92%,2.14%,2506.85,148.34,53.61,36.14%
60,2019,5.03%,1.82%,3230.78,162.35,58.80,36.22%
61,2020,3.68%,1.51%,3756.07,138.12,56.70,41.05%


In [11]:
# we have an issue in that the dataframe doesn't view the first row as the header, so let's fix that
header = table.iloc[0] #grab the first row for the header
table_2 = table[1:] #take the data less the header row
table_2.columns = header #set the header row as the df header

table_2

,Year,Earnings Yield,Dividend Yield,S&P 500,Earnings,Dividends,Payout Ratio
1,1960,5.34%,3.41%,58.11,3.10,1.98,63.86%
2,1961,4.71%,2.85%,71.55,3.37,2.04,60.51%
3,1962,5.81%,3.40%,63.1,3.67,2.15,58.52%
4,1963,5.51%,3.13%,75.02,4.13,2.35,56.81%
5,1964,5.62%,3.05%,84.75,4.76,2.58,54.27%
...,...,...,...,...,...,...,...
58,2017,4.66%,1.83%,2673.61,124.51,48.93,39.30%
59,2018,5.92%,2.14%,2506.85,148.34,53.61,36.14%
60,2019,5.03%,1.82%,3230.78,162.35,58.80,36.22%
61,2020,3.68%,1.51%,3756.07,138.12,56.70,41.05%


In [13]:
# check the datatypes of each column
dataTypeSeries = table_2.dtypes
dataTypeSeries

0
Year              object
Earnings Yield    object
Dividend Yield    object
S&P 500           object
Earnings          object
Dividends         object
Payout  Ratio     object
dtype: object

In [30]:
# They're all objects, we want them as integers
# Edit all the columns that have '%' in them (i.e. remove the %'s), using regex:
table_3 = table_2.replace(to_replace = "\%", value = '', regex = True)

# now that all the cells are clean, format them all as floating point numbers:
table_4 = table_3.astype('float')

# Change "Year" column to integer format
table_4["Year"] = table_4["Year"].astype("int")
table_4

,Year,Earnings Yield,Dividend Yield,S&P 500,Earnings,Dividends,Payout Ratio
1,1960,5.34,3.41,58.11,3.10,1.98,63.86
2,1961,4.71,2.85,71.55,3.37,2.04,60.51
3,1962,5.81,3.40,63.10,3.67,2.15,58.52
4,1963,5.51,3.13,75.02,4.13,2.35,56.81
5,1964,5.62,3.05,84.75,4.76,2.58,54.27
...,...,...,...,...,...,...,...
58,2017,4.66,1.83,2673.61,124.51,48.93,39.30
59,2018,5.92,2.14,2506.85,148.34,53.61,36.14
60,2019,5.03,1.82,3230.78,162.35,58.80,36.22
61,2020,3.68,1.51,3756.07,138.12,56.70,41.05


In [51]:
# Note that, annoyingly, "Payout Ratio" has two spaces between "Payout" and "Ratio", so I'm going to rename the 
table_5 = table_4.rename(columns = {'Payout  Ratio': 'Payout Ratio'})
table_5

,Year,Earnings Yield,Dividend Yield,S&P 500,Earnings,Dividends,Payout Ratio
1,1960,5.34,3.41,58.11,3.10,1.98,63.86
2,1961,4.71,2.85,71.55,3.37,2.04,60.51
3,1962,5.81,3.40,63.10,3.67,2.15,58.52
4,1963,5.51,3.13,75.02,4.13,2.35,56.81
5,1964,5.62,3.05,84.75,4.76,2.58,54.27
...,...,...,...,...,...,...,...
58,2017,4.66,1.83,2673.61,124.51,48.93,39.30
59,2018,5.92,2.14,2506.85,148.34,53.61,36.14
60,2019,5.03,1.82,3230.78,162.35,58.80,36.22
61,2020,3.68,1.51,3756.07,138.12,56.70,41.05


In [52]:
# Divide "Earnings Yield", "Dividend Yield", and "Payout Ratio" columns by 100
table_5['Earnings Yield'] = table_5['Earnings Yield']/100
table_5['Dividend Yield'] = table_5['Dividend Yield']/100
table_5['Payout Ratio'] = table_5['Payout Ratio']/100
table_5

,Year,Earnings Yield,Dividend Yield,S&P 500,Earnings,Dividends,Payout Ratio
1,1960,0.0534,0.0341,58.11,3.10,1.98,0.6386
2,1961,0.0471,0.0285,71.55,3.37,2.04,0.6051
3,1962,0.0581,0.0340,63.10,3.67,2.15,0.5852
4,1963,0.0551,0.0313,75.02,4.13,2.35,0.5681
5,1964,0.0562,0.0305,84.75,4.76,2.58,0.5427
...,...,...,...,...,...,...,...
58,2017,0.0466,0.0183,2673.61,124.51,48.93,0.3930
59,2018,0.0592,0.0214,2506.85,148.34,53.61,0.3614
60,2019,0.0503,0.0182,3230.78,162.35,58.80,0.3622
61,2020,0.0368,0.0151,3756.07,138.12,56.70,0.4105


In [53]:
# export the file to excel:
table_5.to_excel("S&P_Historical_Data.xlsx", sheet_name='Data')

In [54]:
# Download the files to your local machine
from google.colab import files
files.download('S&P_Historical_Data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>